In [48]:
import re
from lightrag.core import Component, Generator
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [31]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""

Let's turn on the library log to help with debugging.

In [32]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [33]:
#Toy example

class DocQA(Component):
    def __init__(self):
        super(DocQA, self).__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        return self.doc(prompt_kwargs={"input_str": query}).data
    

In [34]:
doc = DocQA()
doc

2024-06-09 23:12:34 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
)

In [35]:
print(doc("What is the best treatment for headache?"))

2024-06-09 23:12:34 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?'}
2024-06-09 23:12:34 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 23:12:34 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?'}]}
2024-06-09 23:12:36 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 23:12:36 - INFO - [generator.py:205:call] - output: GeneratorOutput(data="The best treatment for a headache varies depending on the cause of the headache. For tension headaches, over-the-counter pain relievers such as ibuprofen or acetaminophen may be helpful. If the headache is due to dehydration, drinking plenty of water and rest may help. For migraines, prescription medications such as triptans may be necessar

In [36]:
# list other subcomponents

for subcomponent in doc.named_components():
    print(subcomponent)

('', DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
))
('doc', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
))
('doc.prompt', Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str']))
('doc.model_client', OpenAIClient())


Let's add a parameter

In [37]:
from lightrag.core.parameter import Parameter

doc.register_parameter("demo", param=Parameter(data="demo"))

In [38]:
# list all parameters
for param in doc.named_parameters():
    print(param)

('demo', Parameter: demo)


In [39]:
doc.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'doc': {'type': 'Generator',
    'data': {'_components': {'model_client': {'type': 'OpenAIClient',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        'sync_client': <openai.OpenAI at 0x1197c97d0>,
        'async_client': None,
        '_api_key': None}},
      'prompt': {'type': 'Prompt',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        '_template_string': '<SYS> You are a doctor </SYS> User: {{input_str}}',
        'prompt_variables': ['input_str'],
        'preset_prompt_kwargs': None}}},
     '_parameters': {},
     'training': False,
     'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
     'preset_prompt_kwargs': None,
     'model_kwargs': {'model': 'gpt-3.5-turbo'},
     'output_processors': None,
     '_trainable_params': []}}},
  '_parameters': {'demo': {'data': 'demo', 'requires_opt': True}},
  'training': False}}

In [40]:
from utils.serialization import save_json

save_json(doc.to_dict(), "doc.json")

In [41]:
doc.state_dict()

OrderedDict([('demo', Parameter: demo)])

In [42]:
doc.call("What is the best treatment for a cold?")

2024-06-09 23:12:36 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for a cold?'}
2024-06-09 23:12:36 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 23:12:36 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for a cold?'}]}
2024-06-09 23:12:37 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 23:12:37 - INFO - [generator.py:205:call] - output: GeneratorOutput(data="The best treatment for a cold is usually rest, hydration, and over-the-counter medications to relieve symptoms such as fever, congestion, and sore throat. It's also important to wash your hands frequently and practice good hygiene to prevent spreading the cold to others. In some cases, your doctor may recommend antiviral medications or antibiot

"The best treatment for a cold is usually rest, hydration, and over-the-counter medications to relieve symptoms such as fever, congestion, and sore throat. It's also important to wash your hands frequently and practice good hygiene to prevent spreading the cold to others. In some cases, your doctor may recommend antiviral medications or antibiotics if a secondary bacterial infection is present."

In [43]:
from lightrag.core.component import FunComponent

def add_one(x):
    return x + 1

fun_component = FunComponent(add_one)
print(fun_component(1))  
print(type(fun_component))  

# output:
# 2
# <class 'core.component.FunComponent'>

2
<class 'lightrag.core.component.FunComponent'>


In [44]:
from lightrag.core.component import fun_to_component 

fun_component = fun_to_component(add_one)
print(fun_component(1))
print(type(fun_component))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [45]:
# use it as a decorator
@fun_to_component
def add_one(x):
    return x + 1

print(add_one(1))
print(type(add_one))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [46]:
from lightrag.core.component import Sequential

@fun_to_component
def enhance_query(query:str) -> str:
    return query + "Please be concise and only list the top treatments."

seq = Sequential(enhance_query, doc)

query = "What is the best treatment for headache?"
print(seq(query))

2024-06-09 23:12:38 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?Please be concise and only list the top treatments.'}
2024-06-09 23:12:38 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 23:12:38 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?Please be concise and only list the top treatments.'}]}
2024-06-09 23:12:40 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 23:12:40 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='1. Rest and relaxation\n2. Over-the-counter pain relievers (e.g. ibuprofen, acetaminophen)\n3. Hydration\n4. Warm or cold compress\n5. Deep breathing or relaxation techniques', error=None, raw_response='1. Rest and relaxation\n2. Over-the-c

In [47]:
seq

Sequential(
  (0): EnhanceQueryComponent()
  (1): DocQA(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
      (model_client): OpenAIClient()
    )
  )
)

# TODO: LLM for single choices